In [9]:
from selenium import webdriver
from selenium.common import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import json
import time
driver = webdriver.Chrome()
# the URL of the target page to scrape
url = 'https://www.reddit.com/r/investing/'
# connect to the target URL in Selenium
driver.get(url)
#get_url = driver.current_url
#print("The current url is:"+str(get_url))
posts_list = []  
last_height = driver.execute_script("return document.body.scrollHeight") 
posts_count = 100
while posts_count > len(posts_list): 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    #8 Wait for the page to load  
    time.sleep(5)
    #9 Calculate the new height of the page  
    new_height = driver.execute_script("return document.body.scrollHeight")  
    #10 Check if you have reached the bottom of the page  
    if new_height == last_height:  
        break  
    last_height = new_height  
    #11 Extract the data  
    links = driver.find_elements(By.TAG_NAME,'shreddit-post')  
    for link in links:  
        #12 append extracted data 
        l=link.get_attribute("content-href")
        if l not in posts_list:
            posts_list.append(link.get_attribute("content-href")) 
driver.quit()

In [10]:
import time
import datetime
from dateutil import parser
d_l=[]
for i in posts_list:
    try:
        driver = webdriver.Chrome()
        url = i
        driver.get(url)
        title= driver.find_element(By.TAG_NAME,'shreddit-title').get_attribute("title")
        content=driver.find_element("xpath","//div[@slot='text-body']")
        date=driver.find_element(By.TAG_NAME,"time").get_attribute("datetime")
        date=parser.isoparse(date)
        shadow_host=driver.find_element(By.TAG_NAME,"shreddit-post")
        shadow_root = driver.execute_script('return arguments[0].shadowRoot', shadow_host)
        upvote=shadow_root.find_element(By.CSS_SELECTOR,"[class='relative mr-sm ']").find_element(By.TAG_NAME,"faceplate-number").text
        comment_no=shadow_root.find_element(By.CSS_SELECTOR,"[name='comments-action-button']").find_element(By.TAG_NAME,"faceplate-number").text
        try:
            content.find_element(By.TAG_NAME,"button").click()
            content=driver.find_element("xpath","//div[@slot='text-body']").text
        except:
            content=driver.find_element("xpath","//div[@slot='text-body']").text
        #comments
        comments= driver.find_elements(By.TAG_NAME,'shreddit-comment')
        if comments:
            comments=[j.text for j in comments]
        d={"title":title,"date":date,"content":content,"upvote":upvote,"comment_no":comment_no,"comments":comments,"link":i}
        d_l.append(d)
        driver.quit()
    except:
        continue

In [11]:
len(d_l)

103

In [12]:
d_json=json.dumps(d_l,default=str)
d_json

'[{"title": "Daily General Discussion and Advice Thread - September 28, 2023 : r/investing", "date": "2023-09-28 09:01:08.051000+00:00", "content": "Have a general question? Want to offer some commentary on markets? Maybe you would just like to throw out a neat fact that doesn\'t warrant a self post? Feel free to post here!\\nIf your question is \\"I have $10,000, what do I do?\\" or other \\"advice for my personal situation\\" questions, you should include relevant information, such as the following:\\nHow old are you? What country do you live in?\\nAre you employed/making income? How much?\\nWhat are your objectives with this money? (Buy a house? Retirement savings?)\\nWhat is your time horizon? Do you need this money next month? Next 20yrs?\\nWhat is your risk tolerance? (Do you mind risking it at blackjack or do you need to know its 100% safe?)\\nWhat are you current holdings? (Do you already have exposure to specific funds and sectors? Any other assets?)\\nAny big debts (include i

In [13]:
with open("reddit_investing.json", "w") as outfile:
    outfile.write(d_json)